In [1]:
! pip install cohere altair -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import cohere
import pandas as pd
import numpy as np
import altair as alt

api_key = "22GKhTy4iMnY3iw1MNUjzqOBsFLQow31t6eNQi7s"
co = cohere.Client(api_key)

In [3]:
df_orig = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/atis_intents_train.csv',names = ['intent','query'])

df_orig

,intent,query
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...
...,...,...
4829,atis_airfare,what is the airfare for flights from denver t...
4830,atis_flight,do you have any flights from denver to baltim...
4831,atis_airline,which airlines fly into and out of denver
4832,atis_flight,does continental fly from boston to san franc...


In [4]:
df_orig["intent"].unique()

array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
       'atis_ground_service', 'atis_airline', 'atis_abbreviation',
       'atis_quantity'], dtype=object)

In [7]:
sample_classes = ["atis_airfare","atis_airline","atis_ground_service"]

df = df_orig.sample(frac=0.12,random_state = 30)
df = df[df.intent.isin(sample_classes)]
df_orig = df_orig.drop(df.index)
df.reset_index(drop=True,inplace=True)
intents = df["intent"]
df.drop(columns = ["intent"],inplace=True)
df.head()

# 1. Intuition

When you hear about large language models (LLM), probably the first thing that comes to mind is the text generation capability, such as writing an essay or creating a marketing copy.

But another thing you can get is text representation: a set of numbers that represent what the text means, and somehow capture the semantics of the text. These numbers are called text embeddings.

## 1.1 - Turn text into embeddings

In [11]:
def get_embeddings(text,model="embed-english-v3.0",
                   input_type="search_document"):
    output = co.embed(model=model,
                      input_type=input_type,
                      texts=text)
    return output.embeddings

In [12]:
df["query_embeds"] = get_embeddings(df["query"].tolist())
df.head()

,query,query_embeds
0,please give me round trip fares from baltimor...,"[0.029403687, -0.0026416779, -0.008682251, 0.0..."
1,is there ground transportation from atlanta a...,"[0.018722534, 0.016937256, -0.044189453, -0.02..."
2,what are the most expensive first class ticke...,"[-0.0016584396, -0.02659607, 0.018829346, 0.04..."
3,is there limo service at pittsburgh airport,"[0.031158447, 0.0058021545, -0.021347046, -0.0..."
4,what is the schedule of ground transportation...,"[0.0030918121, 0.010940552, -0.064941406, -0.0..."


## 1.2 - Visualize embeddings on a heatmap

In [13]:
from sklearn.decomposition import PCA

def get_pc(arr,n):
    pca = PCA(n_components=n)
    embeds_transform = pca.fit_transform(arr)
    return embeds_transform